# Day10_0: A/B 테스트 이론 - 정답 노트북

이 노트북은 Day10_0 실습 퀴즈의 정답과 상세한 풀이를 제공합니다.

---

In [ ]:
# 필요한 라이브러리 임포트
import numpy as np
import pandas as pd
from scipy import stats
from statsmodels.stats.proportion import proportions_ztest, proportion_effectsize
from statsmodels.stats.power import NormalIndPower, TTestIndPower
import plotly.express as px
import plotly.graph_objects as go

print("라이브러리 임포트 완료")

---

## Q1. 전환율 계산 ⭐

**문제**: 다음 A/B 테스트 데이터에서 각 그룹의 전환율을 계산하세요.

```python
control_visitors = 5000
control_conversions = 150
treatment_visitors = 5000
treatment_conversions = 200
```

**출력**: 각 그룹 전환율(%), 절대적 차이(%p), 상대적 개선율(%)

In [ ]:
# 정답 코드
control_visitors = 5000
control_conversions = 150
treatment_visitors = 5000
treatment_conversions = 200

# 전환율 계산
control_rate = control_conversions / control_visitors * 100
treatment_rate = treatment_conversions / treatment_visitors * 100

# 절대적 차이 (%p)
absolute_diff = treatment_rate - control_rate

# 상대적 개선율 (%)
relative_lift = (treatment_rate - control_rate) / control_rate * 100

print("전환율 분석 결과")
print("=" * 50)
print(f"Control 전환율: {control_rate:.2f}%")
print(f"Treatment 전환율: {treatment_rate:.2f}%")
print(f"\n절대적 차이: {absolute_diff:.2f}%p")
print(f"상대적 개선율 (Lift): {relative_lift:.1f}%")

In [ ]:
# 테스트
assert abs(control_rate - 3.0) < 0.01, "Control 전환율 계산 오류"
assert abs(treatment_rate - 4.0) < 0.01, "Treatment 전환율 계산 오류"
assert abs(absolute_diff - 1.0) < 0.01, "절대적 차이 계산 오류"
assert abs(relative_lift - 33.33) < 0.1, "상대적 개선율 계산 오류"
print("모든 테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
1. 전환율 = (전환 수 / 방문자 수) * 100
2. 절대적 차이 = Treatment 전환율 - Control 전환율 (단위: %p)
3. 상대적 개선율 = (차이 / Control 전환율) * 100 (단위: %)

**핵심 개념**:
- **절대적 차이 (%p)**: 3% → 4%는 1%p 증가
- **상대적 차이 (%)**: 3% → 4%는 33.3% 증가

**대안**:
- pandas DataFrame으로 정리하면 더 깔끔하게 보고서 작성 가능

**흔한 실수**:
- 절대적 차이와 상대적 차이를 혼동 (1%p vs 33%)
- 100을 곱하지 않아 소수점으로 표시

**실무 팁**:
- 보고서에서는 상대적 개선율(Lift)을 주로 사용
- "전환율이 33% 향상되었다"가 "1%p 증가"보다 임팩트 있음

---

## Q2. 무작위 배정 검증 ⭐⭐

**문제**: 1000명의 사용자를 A/B 그룹에 배정하고, 각 그룹의 비율이 약 50:50인지 확인하세요.

In [ ]:
# 정답 코드
np.random.seed(42)
n_users = 1000

# 방법 1: np.random.choice
assignments_choice = np.random.choice(['A', 'B'], size=n_users, p=[0.5, 0.5])
group_a_count = np.sum(assignments_choice == 'A')
group_b_count = np.sum(assignments_choice == 'B')

print("무작위 배정 결과 (np.random.choice)")
print("=" * 50)
print(f"A 그룹: {group_a_count}명 ({group_a_count/n_users*100:.1f}%)")
print(f"B 그룹: {group_b_count}명 ({group_b_count/n_users*100:.1f}%)")

# 방법 2: np.random.binomial (더 효율적)
np.random.seed(42)
group_b_binomial = np.random.binomial(n_users, 0.5)
group_a_binomial = n_users - group_b_binomial

print(f"\n무작위 배정 결과 (np.random.binomial)")
print(f"A 그룹: {group_a_binomial}명")
print(f"B 그룹: {group_b_binomial}명")

In [ ]:
# 테스트: 각 그룹이 40%~60% 범위 내
assert 400 <= group_a_count <= 600, "A 그룹 비율 이상"
assert 400 <= group_b_count <= 600, "B 그룹 비율 이상"
assert group_a_count + group_b_count == n_users, "총 합 오류"
print("모든 테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
- `np.random.choice`: 각 사용자별로 A/B 중 하나 선택
- `np.random.binomial`: n번 시도 중 성공 횟수 (더 효율적)

**핵심 개념**:
- 무작위 배정은 **선택 편향(Selection Bias)** 방지
- 완벽한 50:50은 불가능 - 약간의 변동은 정상

**대안**:
- 해시 기반 배정: 동일 사용자에게 일관된 그룹 배정
- 층화 무작위 배정: 특정 속성별로 균형 맞춤

**흔한 실수**:
- `random.seed()` 없이 매번 다른 결과
- 확률 합이 1이 아닌 경우

**실무 팁**:
- 프로덕션에서는 해시 기반 배정 사용 (사용자 ID → 그룹)
- A/A 테스트로 무작위 배정 시스템 검증 필수

---

## Q3. 실험 기간 계산 ⭐⭐

**문제**: 일일 방문자 5,000명인 사이트에서 각 그룹에 3,000명이 필요할 때, 최소 실험 기간을 계산하세요.

In [ ]:
# 정답 코드
daily_traffic = 5000
required_per_group = 3000

# 전체 트래픽 100% 사용, 50:50 분배
daily_per_group = daily_traffic * 0.5  # 각 그룹 일일 배정

# 필요 일수 계산
days_needed = np.ceil(required_per_group / daily_per_group)

# 최소 7일 (주기성 고려)
min_experiment_days = max(7, int(days_needed))

print("실험 기간 계산")
print("=" * 50)
print(f"일일 방문자: {daily_traffic:,}명")
print(f"일일 각 그룹 배정: {daily_per_group:,.0f}명")
print(f"필요 표본/그룹: {required_per_group:,}명")
print(f"\n필요 일수 (계산): {days_needed:.0f}일")
print(f"최소 실험 기간 (주기성 고려): {min_experiment_days}일")

In [ ]:
# 테스트
assert days_needed >= 1, "일수 계산 오류"
assert min_experiment_days >= 7, "최소 7일 조건 미충족"
print("모든 테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
1. 일일 각 그룹 배정 = 일일 트래픽 * 0.5
2. 필요 일수 = 필요 표본 / 일일 배정 (올림)
3. 최소 7일 보장 (주중/주말 패턴 포함)

**핵심 개념**:
- 주기성: 요일별 트래픽 패턴이 다름
- 최소 1주일 → 이상적으로 2주 권장

**대안**:
- 트래픽 일부만 실험 (예: 20%)면 기간 5배 증가
- 여러 테스트 동시 진행 시 트래픽 분할 고려

**흔한 실수**:
- 올림(ceil) 대신 내림 또는 반올림 사용
- 주기성 무시하고 너무 짧은 기간 설정

**실무 팁**:
- 세일, 공휴일 등 비정상 기간 피하기
- 여유 기간 추가 (예상 기간 + 3-7일)

---

## Q4. 비율 검정 (Z-test) ⭐⭐⭐

**문제**: 다음 A/B 테스트 결과가 통계적으로 유의한지 `proportions_ztest`로 검정하세요.

In [ ]:
# 정답 코드
from statsmodels.stats.proportion import proportions_ztest

control = {'visitors': 8000, 'conversions': 240}  # 3%
treatment = {'visitors': 8000, 'conversions': 320}  # 4%

# 성공 수와 시도 수 배열
successes = np.array([control['conversions'], treatment['conversions']])
trials = np.array([control['visitors'], treatment['visitors']])

# Z-검정 수행
z_stat, p_value = proportions_ztest(successes, trials, alternative='two-sided')

# 결과 출력
print("비율 검정 (Z-test) 결과")
print("=" * 50)
print(f"Control 전환율: {control['conversions']/control['visitors']:.2%}")
print(f"Treatment 전환율: {treatment['conversions']/treatment['visitors']:.2%}")
print(f"\nZ-statistic: {z_stat:.4f}")
print(f"p-value: {p_value:.6f}")

# 결론
alpha = 0.05
if p_value < alpha:
    print(f"\np-value ({p_value:.4f}) < alpha ({alpha})")
    print("결론: 두 그룹 간 전환율 차이가 통계적으로 유의합니다!")
else:
    print(f"\np-value ({p_value:.4f}) >= alpha ({alpha})")
    print("결론: 통계적으로 유의한 차이가 없습니다.")

In [ ]:
# 테스트
assert z_stat < 0, "Z-statistic 부호 오류 (Treatment가 더 높으므로 음수 가능)"
assert p_value < 0.05, "p-value 계산 오류"
print("모든 테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
1. 성공 수(전환)와 시도 수(방문자)를 배열로 준비
2. `proportions_ztest()`로 Z-검정 수행
3. p-value와 유의수준(0.05) 비교

**핵심 개념**:
- Z-검정: 두 비율의 차이가 0인지 검정
- 양측검정(two-sided): 어느 쪽이 더 높은지 모를 때
- 단측검정(greater/less): 방향이 명확할 때

**대안**:
- 카이제곱 검정도 비율 비교에 사용 가능
- 베이지안 A/B 테스트 접근법

**흔한 실수**:
- 배열 순서 혼동 (Control, Treatment)
- 단측/양측 검정 선택 오류

**실무 팁**:
- 대부분의 A/B 테스트는 양측검정 사용
- p-value만 보지 말고 효과 크기도 함께 확인

---

## Q5. 신뢰구간 계산 ⭐⭐⭐

**문제**: Q4의 전환율 차이에 대한 95% 신뢰구간을 계산하세요.

In [ ]:
# 정답 코드
p1 = 240 / 8000  # Control: 0.03
p2 = 320 / 8000  # Treatment: 0.04
n1 = n2 = 8000

# 전환율 차이
diff = p2 - p1

# 표준오차 (Standard Error)
se = np.sqrt(p1 * (1 - p1) / n1 + p2 * (1 - p2) / n2)

# 95% 신뢰구간 (z = 1.96)
z_critical = 1.96
ci_low = diff - z_critical * se
ci_high = diff + z_critical * se

print("신뢰구간 계산 결과")
print("=" * 50)
print(f"Control 전환율: {p1:.2%}")
print(f"Treatment 전환율: {p2:.2%}")
print(f"전환율 차이: {diff:.2%}p")
print(f"표준오차 (SE): {se:.4f}")
print(f"\n95% 신뢰구간: [{ci_low:.2%}p, {ci_high:.2%}p]")

# 해석
if ci_low > 0:
    print("\n신뢰구간이 0을 포함하지 않음")
    print("Treatment가 Control보다 유의하게 높음!")
elif ci_high < 0:
    print("\n신뢰구간이 0을 포함하지 않음")
    print("Treatment가 Control보다 유의하게 낮음!")
else:
    print("\n신뢰구간이 0을 포함함 → 유의한 차이 없음")

In [ ]:
# 테스트
assert ci_low > 0, "신뢰구간 하한이 양수여야 함"
assert ci_low < ci_high, "하한 < 상한 조건"
print("모든 테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
1. 전환율 차이 = p2 - p1
2. 표준오차 = sqrt(p1*(1-p1)/n1 + p2*(1-p2)/n2)
3. 95% CI = 차이 +/- 1.96 * SE

**핵심 개념**:
- 신뢰구간이 0을 포함하지 않으면 유의함
- 95% 신뢰구간 = "100번 반복하면 95번은 실제 값 포함"

**대안**:
- `statsmodels.stats.proportion.confint_proportions_2indep()` 사용
- 부트스트랩 신뢰구간

**흔한 실수**:
- z_critical을 1.96 대신 다른 값 사용
- 표준오차 공식 오류

**실무 팁**:
- 시각화할 때 error bar로 표현
- 신뢰구간 폭이 넓으면 표본 크기 부족 신호

---

## Q6. 표본 크기 계산 ⭐⭐⭐

**문제**: 현재 전환율 2%에서 2.5%로 개선을 감지하려면 각 그룹에 몇 명이 필요한지 계산하세요.

In [ ]:
# 정답 코드
from statsmodels.stats.proportion import proportion_effectsize
from statsmodels.stats.power import NormalIndPower

baseline_rate = 0.02  # 2%
target_rate = 0.025   # 2.5%

# 효과 크기 (Cohen's h) 계산
effect_size = proportion_effectsize(baseline_rate, target_rate)

# Power Analysis
analysis = NormalIndPower()
required_n = analysis.solve_power(
    effect_size=effect_size,
    alpha=0.05,
    power=0.80,
    ratio=1.0,
    alternative='two-sided'
)

required_n = int(np.ceil(required_n))

print("표본 크기 계산 결과")
print("=" * 50)
print(f"기준 전환율: {baseline_rate:.1%}")
print(f"목표 전환율: {target_rate:.1%}")
print(f"최소 탐지 효과 (MDE): {target_rate - baseline_rate:.1%}p")
print(f"효과 크기 (Cohen's h): {abs(effect_size):.4f}")
print(f"\n유의수준 (alpha): 0.05")
print(f"검정력 (power): 0.80")
print(f"\n필요 표본 크기: 각 그룹 {required_n:,}명")
print(f"총 필요 표본: {required_n * 2:,}명")

In [ ]:
# 테스트
assert required_n > 1000, "표본 크기가 너무 작음"
assert required_n < 50000, "표본 크기가 비정상적으로 큼"
print(f"필요 표본 크기: {required_n:,}명/그룹 - 테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
1. Cohen's h로 효과 크기 계산
2. NormalIndPower로 필요 표본 크기 계산
3. 올림으로 정수 변환

**핵심 개념**:
- MDE가 작을수록 더 많은 표본 필요
- 검정력 80%가 일반적 기준

**대안**:
- 온라인 표본 크기 계산기 (Evan Miller 등)
- 베이지안 접근법 (고정 표본 불필요)

**흔한 실수**:
- 비율 대신 퍼센트 값 입력 (2 대신 0.02)
- alternative 파라미터 누락

**실무 팁**:
- 실험 전 Power Analysis 필수
- 여유 있게 +20% 표본 확보 권장

---

## Q7. 검정력 계산 ⭐⭐⭐

**문제**: 각 그룹 5,000명으로 3% → 3.5% 개선을 감지할 수 있는 검정력을 계산하세요.

In [ ]:
# 정답 코드
n_per_group = 5000
p1 = 0.03   # 3%
p2 = 0.035  # 3.5%

# 효과 크기 계산
effect_size = proportion_effectsize(p1, p2)

# 검정력 계산
analysis = NormalIndPower()
power = analysis.solve_power(
    effect_size=effect_size,
    nobs1=n_per_group,
    alpha=0.05,
    ratio=1.0,
    alternative='two-sided'
)

print("검정력 분석 결과")
print("=" * 50)
print(f"기준 전환율: {p1:.1%}")
print(f"목표 전환율: {p2:.1%}")
print(f"효과 크기 (Cohen's h): {abs(effect_size):.4f}")
print(f"각 그룹 표본 크기: {n_per_group:,}명")
print(f"\n검정력 (Statistical Power): {power:.1%}")

# 해석
if power >= 0.80:
    print("\n충분한 검정력 확보 (80% 이상)")
else:
    print(f"\n검정력 부족! 80% 달성을 위해 더 많은 표본 필요")
    required = analysis.solve_power(
        effect_size=effect_size,
        alpha=0.05,
        power=0.80,
        ratio=1.0,
        alternative='two-sided'
    )
    print(f"권장 표본 크기: {int(np.ceil(required)):,}명/그룹")

In [ ]:
# 테스트
assert 0 < power < 1, "검정력 범위 오류"
print(f"검정력: {power:.1%} - 테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
1. 효과 크기 계산
2. solve_power()에 nobs1 지정하여 power 계산

**핵심 개념**:
- 검정력 = 실제 효과가 있을 때 탐지할 확률
- 80% 미만이면 Type II 오류 위험

**대안**:
- 검정력 곡선으로 시각화
- 다양한 표본 크기에 대해 비교

**흔한 실수**:
- solve_power 파라미터 혼동
- ratio 파라미터 누락 (1:1 비율)

**실무 팁**:
- 검정력 80% 이상 확보 후 실험 시작
- 검정력이 낮으면 MDE를 늘리거나 표본 크기 증가

---

## Q8. A/B 테스트 결과 시각화 ⭐⭐⭐⭐

**문제**: A/B 테스트 결과를 Plotly 막대 그래프로 시각화하세요. 신뢰구간(error bar) 포함.

In [ ]:
# 정답 코드
results = {
    '그룹': ['Control', 'Treatment'],
    '전환율': [2.5, 3.2],
    'CI_error': [0.3, 0.35]
}

# Plotly 막대 그래프
fig = go.Figure()

fig.add_trace(go.Bar(
    x=results['그룹'],
    y=results['전환율'],
    error_y=dict(
        type='data',
        array=results['CI_error'],
        visible=True,
        color='black',
        thickness=2,
        width=10
    ),
    marker_color=['#3498db', '#2ecc71'],
    text=[f'{r:.1f}%' for r in results['전환율']],
    textposition='outside',
    textfont=dict(size=14)
))

# 레이아웃 설정
fig.update_layout(
    title=dict(
        text='A/B 테스트 결과: 전환율 비교 (95% 신뢰구간)',
        font=dict(size=16)
    ),
    xaxis_title='그룹',
    yaxis_title='전환율 (%)',
    yaxis=dict(range=[0, 4.5]),
    showlegend=False,
    width=600,
    height=450,
    bargap=0.4
)

# 상대적 개선율 표시
lift = (results['전환율'][1] - results['전환율'][0]) / results['전환율'][0] * 100
fig.add_annotation(
    x=0.5, y=4.0,
    text=f"Lift: +{lift:.1f}%",
    showarrow=False,
    font=dict(size=14, color='green')
)

fig.show()

### 💡 풀이 설명

**접근 방법**:
1. `go.Bar()` 로 막대 그래프 생성
2. `error_y` 파라미터로 신뢰구간 표시
3. annotation으로 Lift 표시

**핵심 개념**:
- Error bar는 불확실성을 시각화
- Error bar가 겹치지 않으면 유의한 차이

**대안**:
- `px.bar()` 사용 (더 간단)
- 신뢰구간을 별도 선으로 표시

**흔한 실수**:
- error_y 형식 오류
- y축 범위 미설정으로 text 잘림

**실무 팁**:
- 색상으로 Control/Treatment 구분
- Lift와 p-value 함께 표시

---

## Q9. 세그먼트별 분석 ⭐⭐⭐⭐

**문제**: 세그먼트별(모바일/데스크톱)과 전체로 분석하여 Simpson's Paradox가 있는지 확인하세요.

In [ ]:
# 정답 코드
segment_data = {
    'segment': ['Mobile', 'Mobile', 'Desktop', 'Desktop'],
    'group': ['A', 'B', 'A', 'B'],
    'visitors': [6000, 4000, 4000, 6000],
    'conversions': [180, 160, 200, 240]
}

df = pd.DataFrame(segment_data)
df['conversion_rate'] = df['conversions'] / df['visitors'] * 100

print("세그먼트별 전환율 분석")
print("=" * 60)

# 세그먼트별 분석
for segment in ['Mobile', 'Desktop']:
    seg_df = df[df['segment'] == segment]
    a_rate = seg_df[seg_df['group'] == 'A']['conversion_rate'].values[0]
    b_rate = seg_df[seg_df['group'] == 'B']['conversion_rate'].values[0]
    winner = 'A' if a_rate > b_rate else 'B' if b_rate > a_rate else 'Tie'
    print(f"\n{segment}:")
    print(f"  A: {a_rate:.2f}%, B: {b_rate:.2f}% → Winner: {winner}")

# 전체 분석
total_a = df[df['group'] == 'A'].agg({'visitors': 'sum', 'conversions': 'sum'})
total_b = df[df['group'] == 'B'].agg({'visitors': 'sum', 'conversions': 'sum'})

overall_a = total_a['conversions'] / total_a['visitors'] * 100
overall_b = total_b['conversions'] / total_b['visitors'] * 100
overall_winner = 'A' if overall_a > overall_b else 'B' if overall_b > overall_a else 'Tie'

print(f"\n전체:")
print(f"  A: {overall_a:.2f}%, B: {overall_b:.2f}% → Winner: {overall_winner}")

# Simpson's Paradox 확인
print("\n" + "=" * 60)
mobile_winner = 'A' if df[(df['segment']=='Mobile') & (df['group']=='A')]['conversion_rate'].values[0] > \
                       df[(df['segment']=='Mobile') & (df['group']=='B')]['conversion_rate'].values[0] else 'B'
desktop_winner = 'A' if df[(df['segment']=='Desktop') & (df['group']=='A')]['conversion_rate'].values[0] > \
                        df[(df['segment']=='Desktop') & (df['group']=='B')]['conversion_rate'].values[0] else 'B'

if (mobile_winner == desktop_winner) and (mobile_winner != overall_winner):
    print("Simpson's Paradox 발견!")
    print(f"세그먼트별로는 {mobile_winner}가 좋지만, 전체로는 {overall_winner}가 좋아 보임")
else:
    print("Simpson's Paradox 없음")
    print("세그먼트별 결과와 전체 결과가 일관됨")

### 💡 풀이 설명

**접근 방법**:
1. 세그먼트별 전환율 계산
2. 전체 합산 후 전체 전환율 계산
3. 세그먼트별 승자와 전체 승자 비교

**핵심 개념**:
- Simpson's Paradox: 세그먼트별 결과와 전체 결과가 다름
- 원인: 세그먼트 크기(비율)가 그룹간 다름

**대안**:
- 가중 평균으로 조정된 비교
- 층화 분석

**흔한 실수**:
- 전체 평균을 단순 평균으로 계산 (가중 평균 필요)
- 세그먼트 크기 차이 무시

**실무 팁**:
- 항상 주요 세그먼트별로 분석
- 전체 결과만 보면 함정에 빠질 수 있음

---

## Q10. 종합 A/B 테스트 분석 ⭐⭐⭐⭐⭐

**문제**: 마케팅 캠페인 A/B 테스트 데이터를 완전히 분석하세요.

In [ ]:
# 정답 코드
campaign_test = {
    'Control': {'visitors': 12000, 'clicks': 600, 'conversions': 120, 'revenue': 24000000},
    'Treatment': {'visitors': 12000, 'clicks': 840, 'conversions': 168, 'revenue': 30240000}
}

# 1. 지표 계산
metrics = {}
for group, data in campaign_test.items():
    metrics[group] = {
        'CTR': data['clicks'] / data['visitors'] * 100,
        '전환율': data['conversions'] / data['visitors'] * 100,
        'AOV': data['revenue'] / data['conversions'],
        'ARPU': data['revenue'] / data['visitors']
    }

print("=" * 70)
print("A/B 테스트 종합 분석 리포트")
print("=" * 70)

print("\n1. 핵심 지표 비교")
print("-" * 50)
metrics_df = pd.DataFrame(metrics).T
print(metrics_df.round(2).to_string())

# 2. 통계적 유의성 검정 (전환율)
print("\n2. 통계적 유의성 검정 (전환율)")
print("-" * 50)

successes = np.array([campaign_test['Control']['conversions'], 
                      campaign_test['Treatment']['conversions']])
trials = np.array([campaign_test['Control']['visitors'], 
                   campaign_test['Treatment']['visitors']])

z_stat, p_value = proportions_ztest(successes, trials, alternative='two-sided')
print(f"Z-statistic: {z_stat:.4f}")
print(f"p-value: {p_value:.6f}")

# 3. 95% 신뢰구간
print("\n3. 전환율 차이의 95% 신뢰구간")
print("-" * 50)

p1 = campaign_test['Control']['conversions'] / campaign_test['Control']['visitors']
p2 = campaign_test['Treatment']['conversions'] / campaign_test['Treatment']['visitors']
n1 = n2 = 12000

diff = p2 - p1
se = np.sqrt(p1*(1-p1)/n1 + p2*(1-p2)/n2)
ci_low = diff - 1.96 * se
ci_high = diff + 1.96 * se

print(f"전환율 차이: {diff:.2%}p")
print(f"95% 신뢰구간: [{ci_low:.2%}p, {ci_high:.2%}p]")

# 4. 상대적 개선율 (Lift)
print("\n4. 상대적 개선율 (Lift)")
print("-" * 50)

ctr_lift = (metrics['Treatment']['CTR'] - metrics['Control']['CTR']) / metrics['Control']['CTR'] * 100
conv_lift = (metrics['Treatment']['전환율'] - metrics['Control']['전환율']) / metrics['Control']['전환율'] * 100
arpu_lift = (metrics['Treatment']['ARPU'] - metrics['Control']['ARPU']) / metrics['Control']['ARPU'] * 100

print(f"CTR Lift: {ctr_lift:+.1f}%")
print(f"전환율 Lift: {conv_lift:+.1f}%")
print(f"ARPU Lift: {arpu_lift:+.1f}%")

# 5. 비즈니스 인사이트
print("\n" + "=" * 70)
print("5. 비즈니스 인사이트 및 권장사항")
print("=" * 70)

if p_value < 0.05:
    print("\n[결론] Treatment 그룹이 통계적으로 유의하게 우수합니다!")
    print(f"\n- 전환율 {conv_lift:.1f}% 향상 (p < 0.001)")
    print(f"- 추가 수익: {(campaign_test['Treatment']['revenue'] - campaign_test['Control']['revenue']):,}원")
    print(f"\n[권장사항]")
    print("1. Treatment 캠페인을 전체 사용자에게 적용")
    print("2. 세그먼트별 분석으로 효과가 더 큰 그룹 파악")
    print("3. 후속 테스트로 추가 최적화 진행")
else:
    print("\n[결론] 두 그룹 간 유의한 차이가 없습니다.")
    print("\n[권장사항]")
    print("1. 실험 기간 연장 또는 표본 크기 증가")
    print("2. 다른 변수 테스트 고려")

In [ ]:
# 시각화
fig = make_subplots(rows=1, cols=2, subplot_titles=('전환율 비교', 'ARPU 비교'))

# 전환율 비교
fig.add_trace(
    go.Bar(x=['Control', 'Treatment'], 
           y=[metrics['Control']['전환율'], metrics['Treatment']['전환율']],
           marker_color=['#3498db', '#2ecc71'],
           text=[f"{v:.2f}%" for v in [metrics['Control']['전환율'], metrics['Treatment']['전환율']]],
           textposition='outside'),
    row=1, col=1
)

# ARPU 비교
fig.add_trace(
    go.Bar(x=['Control', 'Treatment'], 
           y=[metrics['Control']['ARPU'], metrics['Treatment']['ARPU']],
           marker_color=['#3498db', '#2ecc71'],
           text=[f"{v:,.0f}" for v in [metrics['Control']['ARPU'], metrics['Treatment']['ARPU']]],
           textposition='outside'),
    row=1, col=2
)

fig.update_layout(
    title='A/B 테스트 결과 시각화',
    showlegend=False,
    height=400,
    width=800
)

fig.show()

In [ ]:
# 테스트
assert p_value < 0.05, "통계적 유의성 검정 오류"
assert conv_lift > 0, "전환율 Lift 계산 오류"
print("종합 분석 테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
1. CTR, 전환율, AOV, ARPU 계산
2. proportions_ztest로 유의성 검정
3. 신뢰구간 수동 계산
4. Lift 계산
5. 비즈니스 인사이트 도출

**핵심 개념**:
- 종합 분석은 여러 지표를 함께 봐야 함
- 통계적 유의성 + 실무적 의미 모두 중요

**대안**:
- 베이지안 A/B 테스트
- 순차 검정 (Sequential Testing)

**흔한 실수**:
- 하나의 지표만 보고 결론
- 통계적 유의성만 보고 실무적 의미 무시

**실무 팁**:
- 보고서에는 주요 지표 + p-value + Lift + 권장사항 포함
- 시각화로 직관적 이해 제공
- 추가 수익/비용 환산으로 비즈니스 임팩트 강조

---

## 학습 완료!

이 정답 노트북을 통해 A/B 테스트의 핵심 개념을 복습했습니다.

**핵심 포인트 정리**:
1. 전환율 = 전환 / 방문자 (절대적 vs 상대적 차이 구분)
2. proportions_ztest()로 비율 차이 검정
3. Power Analysis로 필요 표본 크기 사전 계산
4. Simpson's Paradox 방지를 위해 세그먼트별 분석
5. 통계적 유의성 + 비즈니스 임팩트 함께 고려